<a href="https://colab.research.google.com/github/1900690/syakuyaku-freshness-retention-test/blob/main/syakuyaku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ドライブマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#AVIファイルをmp4に変換する

In [ ]:
#ファイルコピー
import shutil
shutil.copyfile("/content/drive/MyDrive/TLC00001.AVI", "/content/TLC00001.AVI")

'/content/TLC00001.AVI'

In [1]:
!pip install opencv-python
!pip install opencv-contrib-python

In [5]:
#ライブラリのインポート
import cv2

#VideoCaptureオブジェクトを取得
cap = cv2.VideoCapture('/content/TLC00007.AVI')

#動画のプロパティを取得
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

#書き出し設定
fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
writer = cv2.VideoWriter('/content/TLC00007.mp4', fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    writer.write(frame)
    if not ret:
        break

writer.release()
cap.release()

In [ ]:
#ファイルコピー
import shutil
shutil.copyfile("/content/TLC00004.mp4", "/content/drive/MyDrive/TLC00004.mp4")

'/content/drive/MyDrive/TLC00004.mp4'

#動画から画像を取り出す

In [ ]:
import shutil
shutil.rmtree("/content/slice")

In [14]:
import cv2
import os



LOAD_FOLDA = '/content/bunnkatuyou'


def save_frame_range(video_path,
                     dir_path, ext='jpg'):

    basename=video_path[-12:-4]
    cap = cv2.VideoCapture(video_path)

    # 総フレーム数
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT));
    print(total_frames)

    #真ん中+1のフレームを取り出すプラスの数値を調整することでフレームをずらせる
    center_frame=total_frames
    start_frame=0#center_frame
    stop_frame=total_frames
    step_frame=1#center_frame

    if not cap.isOpened():
        return

    os.makedirs(dir_path, exist_ok=True)
    base_path = os.path.join(dir_path, basename)

    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

    for n in range(start_frame, stop_frame, step_frame):
        cap.set(cv2.CAP_PROP_POS_FRAMES, n)
        ret, frame = cap.read()
        if ret:
            cv2.imwrite('{}_{}.{}'.format(base_path, str(n).zfill(digit), ext), frame)
        else:
            return

#スライスフォルダを作成
SAVE_NAME = 'slice'
if not os.path.exists('./' +SAVE_NAME):
    os.mkdir('./' +SAVE_NAME)

LOAD_NAME = os.listdir(LOAD_FOLDA)


for file_name in LOAD_NAME:
  IMAGE_PATH= LOAD_FOLDA +'/'+file_name
  save_frame_range(IMAGE_PATH,
                 '/content/'+SAVE_NAME)

image_date=LOAD_FOLDA[-8:]

0
1
1


In [ ]:
#画像をまとめてダウンロード
from google.colab import files
import shutil

shutil.make_archive('slice', format='zip', root_dir='/content/slice')

files.download('slice.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#魚眼補正

[参考](https://qiita.com/hiro_o_tama/items/cb544fe64ca373750aae)

In [ ]:
import cv2
# assert cv2.__version__[0] == '3', 'The fisheye module requires opencv version >= 3.0.0'
import numpy as np
import os
import glob

CHECKERBOARD = (6,9)

print(CHECKERBOARD)
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_CHECK_COND+cv2.fisheye.CALIB_FIX_SKEW
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
_img_shape = None
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('source/*.jpg')
for fname in images:
    img = cv2.imread(fname)
    if _img_shape == None:
        _img_shape = img.shape[:2]
    else:
        assert _img_shape == img.shape[:2], "All images must share the same size."
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)

print('img end')
N_OK = len(objpoints)
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
rms, _, _, _, _ = \
    cv2.fisheye.calibrate(
        objpoints,
        imgpoints,
        gray.shape[::-1],
        K,
        D,
        rvecs,
        tvecs,
        calibration_flags,
        (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
    )
print("Found " + str(N_OK) + " valid images for calibration")
print("DIM=" + str(_img_shape[::-1]))
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")



In [ ]:
import numpy as np
import cv2
import sys
import os

# You should replace these 3 lines with the output in calibration step
DIM=##
K=##
D=##

def undistort(img_path):
    img = cv2.imread(img_path)
    h,w = img.shape[:2]
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, DIM, cv2.CV_16SC2)
    undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    # cv2.imshow("undistorted", undistorted_img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    cv2.imwrite('out_' + os.path.basename(img_path), undistorted_img)
if __name__ == '__main__':
    for p in sys.argv[1:]:
        undistort(p)


#複数クラス検出

In [ ]:
#@title グーグルドライブに接続

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title ultralytics(AIライブラリ)をインストール
%pip -q install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.112 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.3/78.2 GB disk)


In [ ]:
#@title 動画で検証

#作業ディレクトリを移動
%cd '/content/drive/MyDrive/pest-object-detection'

!yolo task=detect \
      mode=predict \
      model=/content/drive/MyDrive/pest-object-detection/runs/detect/train9xbest/weights/best.pt \
      conf=0.4 \
      imgsz=1280\
      show_conf=False \
      save_txt=True \
      device=0\
      source="/content/drive/MyDrive/TLC00004.mp4"

/content/drive/MyDrive/pest-object-detection
Ultralytics YOLOv8.0.112 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68129346 parameters, 0 gradients

video 1/1 (1/525) /content/drive/MyDrive/TLC00004.mp4: 736x1280 34 Es, 25 As, 127.5ms
video 1/1 (2/525) /content/drive/MyDrive/TLC00004.mp4: 736x1280 34 Es, 24 As, 117.6ms
video 1/1 (3/525) /content/drive/MyDrive/TLC00004.mp4: 736x1280 35 Es, 24 As, 116.8ms
video 1/1 (4/525) /content/drive/MyDrive/TLC00004.mp4: 736x1280 34 Es, 24 As, 117.1ms
video 1/1 (5/525) /content/drive/MyDrive/TLC00004.mp4: 736x1280 32 Es, 24 As, 117.0ms
video 1/1 (6/525) /content/drive/MyDrive/TLC00004.mp4: 736x1280 33 Es, 24 As, 117.5ms
video 1/1 (7/525) /content/drive/MyDrive/TLC00004.mp4: 736x1280 33 Es, 24 As, 119.8ms
video 1/1 (8/525) /content/drive/MyDrive/TLC00004.mp4: 736x1280 33 Es, 25 As, 117.2ms
video 1/1 (9/525) /content/drive/MyDrive/TLC00004.mp4: 736x1280 31 Es, 24 As, 118.4ms
video 1/1 (10/525) /cont

In [ ]:
#@title 圧縮してダウンロード
#ライブラリをインポート
import shutil
import os
from google.colab import files

#スイッチ
ファイルをグーグルドライブに保存=False#@param {type:"boolean"}
drive=ファイルをグーグルドライブに保存
ファイルをダウンロードする=True#@param {type:"boolean"}
direct=ファイルをダウンロードする

#圧縮
shutil.make_archive('/content/predict', format='zip', root_dir='/content/drive/MyDrive/pest-object-detection/runs/detect/predict3')
if drive==True:
  #グーグルドライブに接続
  from google.colab import drive
  drive.mount('/content/drive')
  #グーグルドライブにコピー
  shutil.copyfile("/content/datasets.zip","/content/drive/MyDrive/datasets.zip")

if direct==True:
  files.download("/content/predict.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#追跡検出

In [ ]:
#@title グーグルドライブに接続

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title ultralytics(AIライブラリ)をインストール
%pip -q install ultralytics
import ultralytics
ultralytics.checks()

In [ ]:
#@title 検出
from ultralytics import YOLO

#作業ディレクトリを移動
%cd '/content/drive/MyDrive/flower'

# Load a model
model = YOLO('/content/drive/MyDrive/flower/runs/detect/train/weights/best.pt')  # load a custom model

# Track with the model
results = model.track(source="/content/drive/MyDrive/TLC00004.mp4",show_conf=False,save_txt=True,save=True,stream = False,tracker="bytetrack.yaml")

/content/drive/MyDrive/flower




    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/525) /content/drive/MyDrive/TLC00004.mp4: 736x1280 59 flowers, 134.7ms
video 1/1 (2/525) /content/drive/MyDrive/TLC00004.mp4: 736x1280 58 flowers, 126.5ms
video 1/1 (3/525) /content/drive/MyDrive/TLC00004.mp4: 736x1280 58 flowers, 128.1ms
video 1/1 (4/525) /content/drive/MyDrive/TLC00004.mp4: 736x1280 58 flowers, 127.5ms
video 1/1 (5/525) /content/drive/MyDrive/TLC00004.mp4: 736x1280 58 flowers, 127.9ms
video 1/1 (6/525) /content/drive/MyDriv

In [ ]:
#@title 保存
#ライブラリをインポート
import shutil
import os
from google.colab import files

#スイッチ
ファイルをグーグルドライブに保存=False#@param {type:"boolean"}
drive=ファイルをグーグルドライブに保存
ファイルをダウンロードする=True#@param {type:"boolean"}
direct=ファイルをダウンロードする

#作業ディレクトリを移動
%cd '/content'

#圧縮
shutil.make_archive('/content/datasets', format='zip', root_dir='/content/drive/MyDrive/flower/runs/detect/track/labels')
if drive==True:
  #グーグルドライブに接続
  from google.colab import drive
  drive.mount('/content/drive')
  #グーグルドライブにコピー
  shutil.copyfile("/content/datasets.zip","/content/drive/MyDrive/datasets.zip")

if direct==True:
  files.download("/content/datasets.zip")

/content


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#データ分析

追跡のテキストファイルと、クラス分けのテキストファイルの座標から二つのデータの関係性を見出す


*   ノイズの処理をどうするかが肝、平均を出せば処理は可能
*   データの取り出しポイントは5月13日0時～5月23日0時まで11フレーム

In [ ]:
#@title classエクセルアップロード
#ライブラリをインポート(機能を使えるようにする)
import shutil
import os
import glob
import pandas as pd

#データを入れるためのフォルダを作成（すでにフォルダがある場合は消去する）
if os.path.exists('/content/class'):
  shutil.rmtree('/content/class')
if not os.path.exists('/content/class'):
    os.makedirs('/content/class')

#カレントディレクトリを移動
%cd -q '/content/class/'

#データをアップロード
from google.colab import files
uploaded = files.upload()

#元のディレクトリに戻る
%cd -q '/content/'

#pamdasよみこみ
class_list = pd.read_csv('/content/class/annotations.csv')

Saving annotations.csv to annotations.csv


In [ ]:
#@title trackエクセルアップロード
#ライブラリをインポート(機能を使えるようにする)
import shutil
import os
import glob
import pandas as pd


#データを入れるためのフォルダを作成（すでにフォルダがある場合は消去する）
if os.path.exists('/content/track'):
  shutil.rmtree('/content/track')
if not os.path.exists('/content/track'):
    os.makedirs('/content/track')

#カレントディレクトリを移動
%cd -q '/content/track/'

#データをアップロード
from google.colab import files
uploaded = files.upload()

#元のディレクトリに戻る
%cd -q '/content/'

#pamdasよみこみ
track_list = pd.read_csv('/content/track/annotations.csv')

Saving annotations.csv to annotations.csv


In [ ]:
#@title 処理
#classを画像ごとに分割
imagename_class = class_list['Name'].unique()
for name in imagename_class:
  class_sprit_name=os.path.splitext(os.path.basename(name))[0]
  class_sprit=class_list[class_list['Name'] == name]
  class_sprit.to_csv('/content/class/'+class_sprit_name+'.csv', index=False)


#trackを画像ごとに分割
imagename_track = track_list['Name'].unique()
for name in imagename_track:
  track_sprit_name=os.path.splitext(os.path.basename(name))[0]
  track_sprit=track_list[track_list['Name'] == name]
  track_sprit.to_csv('/content/track/'+track_sprit_name+'.csv', index=False)

#面積を計算し、面積順で並べ替えし、ランクを数値に直す
for name in imagename_class:
  class_sprit_name=os.path.splitext(os.path.basename(name))[0]
  img = pd.read_csv('/content/class/'+class_sprit_name+'.csv')
  img['rotation'] = (img['x2']-img['x1'])*(img['y2']-img['y1'])
  img2 = img.sort_values('rotation', ascending=False)
  img2.Value = img2.Value.replace("A", 2)
  img2.Value = img2.Value.replace("B", 3)
  img2.Value = img2.Value.replace("C", 4)
  img2.Value = img2.Value.replace("D", 5)
  img2.Value = img2.Value.replace("E", 6)
  img2.Value = img2.Value.replace("X", 7)
  img2.to_csv('/content/class/'+class_sprit_name+'.csv', index=False)

#ランクを画像に出力
from PIL import Image, ImageDraw

im = Image.new("L",(1280, 720),(0))
draw = ImageDraw.Draw(im)

for name in imagename_class:
  class_sprit_name=os.path.splitext(os.path.basename(name))[0]
  img = pd.read_csv('/content/class/'+class_sprit_name+'.csv')
  row_number=img.shape[0]
  for number in range(row_number):
    draw.rectangle((img['x1'][number], img['y1'][number], img['x2'][number], img['y2'][number]), fill=(int(img['Value'][number])))
  im.save('/content/class/'+class_sprit_name+'.jpg', quality=95)

#trackの中心点を抽出
for name in imagename_track:
  track_sprit_name=os.path.splitext(os.path.basename(name))[0]
  img = pd.read_csv('/content/track/'+track_sprit_name+'.csv')
  img['rotation'] =(img['x2']-img['x1'])/2+img['x1']
  img['Attributes'] =(img['y2']-img['y1'])/2+img['y1']
  img.to_csv('/content/track/'+track_sprit_name+'.csv', index=False)

#画像からランクを抽出
import numpy as np
for name in imagename_class:
  class_sprit_name=os.path.splitext(os.path.basename(name))[0]
  image = np.array(Image.open('/content/class/'+class_sprit_name+'.jpg'))
  img = pd.read_csv('/content/track/'+class_sprit_name+'.csv')
  for number in range(img.shape[0]):
    img['Type'][number]=image[int(img['Attributes'][number]),int(img['rotation'][number])]
  img.to_csv('/content/track/'+class_sprit_name+'.csv', index=False)

#ランクに戻す
for name in imagename_track:
  track_sprit_name=os.path.splitext(os.path.basename(name))[0]
  img = pd.read_csv('/content/track/'+track_sprit_name+'.csv')
  img.Type = img.Type.replace(2,"A")
  img.Type = img.Type.replace(3,"B")
  img.Type = img.Type.replace(4,"C")
  img.Type = img.Type.replace(5,"D")
  img.Type = img.Type.replace(6,"E")
  img.Type = img.Type.replace(7,"X")
  img.to_csv('/content/track/'+track_sprit_name+'.csv', index=False)

#余計なものを消去して抽出
import numpy as np
import csv
import os

if os.path.exists('/content/result.csv'):
  os.remove('/content/result.csv')


data = [["Name","Type","Value","x1","y1","x2","y2"],]
with open("/content/result.csv", "w") as file:
        writer = csv.writer(file)
        writer.writerows(data)

for name in imagename_track:
  img = pd.read_csv('/content/result.csv')
  track_sprit_name=os.path.splitext(os.path.basename(name))[0]
  img2 = pd.read_csv('/content/track/'+track_sprit_name+'.csv')
  img3=img2[['Name','Type','Value','x1','y1','x2','y2']]
  img4=pd.concat([img, img3], axis=0)
  img4.to_csv('/content/result.csv', index=False)

import shutil

#データを入れるためのフォルダを作成（すでにフォルダがある場合は消去する）
if os.path.exists('/content/result'):
  shutil.rmtree('/content/result')
if not os.path.exists('/content/result'):
    os.makedirs('/content/result')

result_list = pd.read_csv('/content/result.csv')
imagename_result = result_list['Name'].unique()
for name in imagename_result:
  result_sprit_name=os.path.splitext(os.path.basename(name))[0]
  result_sprit=result_list[result_list['Name'] == name]
  result_sprit.to_csv('/content/result/'+result_sprit_name+'.csv', index=False)

#圧縮
shutil.make_archive('/content/result', format='zip', root_dir='/content/result/')

#ダウンロード
files.download("/content/result.zip")